In [1]:
import pandas as pd
import os
from os.path import join
import stanza
import re
from collections import Counter
from langdetect import detect
import pandas as pd
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer 
ana = SentimentIntensityAnalyzer()
import requests
from langdetect import detect
import nltk
from bs4 import BeautifulSoup
from nltk import word_tokenize,pos_tag
import matplotlib.pyplot as plt
import seaborn as sns
import random
from bs4.element import Comment
# import credentials
# key = credentials.key

import string
from nltk.corpus import stopwords
stopword_list = stopwords.words('english')
for mark in string.punctuation + '“”’—‘':
    stopword_list.append(mark)

def remove_punctuation(words):
    new_list= []
    for w in words:
        if w.isalnum():
            new_list.append( w )
    return new_list

def sorted_by_value( dict , ascending = True ):
    if ascending: 
        return {k: v for k, v in sorted(dict.items(), key=lambda item: item[1])}
    else:
        return {k: v for k, v in reversed( sorted(dict.items(), key=lambda item: item[1]))}
    
user_agents = [
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36',
    'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.1 Safari/605.1.15',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 13_1) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.1 Safari/605.1.15'
]

def select_agent():
    n=random.randint(0,len(user_agents)-1)
    return user_agents[n]

def tag_visible(element):
    if element.parent.name in ['style', 'script', 'head', 'title', 'meta', '[document]']:
        return False
    if isinstance(element, Comment):
        return False
    return True

api_key = 'QypW6JzQ4XfGViss1Ks4g'

In [2]:
out_dir = 'Goodreads_reviews'
if not os.path.exists(out_dir):
    os.makedirs(directory)

In [3]:
books = dict()

directory = 'Lists'
files = os.listdir(directory)
files = [file for file in files if not(re.search(r'^[.]',file))]

for list in files:

    path = os.path.join(directory,list)
    df = pd.read_csv(path,sep='\t')
    for i,row in df.iterrows():
        books[row['primary_isbn13']]=(row['author'],'M')
print(len(books))

182


In [7]:

for isbn in books:
    path = join(out_dir,f'reviews_{isbn}.txt')
    if not os.path.exists(path):
        out = open(path,'w',encoding='utf-8')
        
        api_call = 'https://www.goodreads.com/book/isbn/'
        api_call += f'?key={api_key}&id={isbn}&format=json&text_only=true'
        headers = {'User-Agent': select_agent() }
        print(api_call)
        response = requests.get( api_call,headers=headers)
        json_data = response.json()

        reviews_widget = json_data['reviews_widget']

        soup = BeautifulSoup( reviews_widget ,"lxml")

        iframe = soup.find("iframe")

        pages_left = True

        url = iframe.get("src")

        i = 1
        while pages_left == True:
            if re.search( r'[&]page',url,re.IGNORECASE):
                url = url[:url.index('&page')]
            url += '&page=' + str(i)
            print(f" {url}")

            headers = {'User-Agent': select_agent() }
            response_reviews = requests.get(url,headers=headers)
            html_page = response_reviews.text
            
            
            if re.search(r'No\s+reviews\sfound',html_page,re.IGNORECASE):
                pages_left = False
            elif i == 100:
                pages_left = False


            soup = BeautifulSoup( html_page ,"lxml")

            review_links = soup.find_all('a',{'class':'gr_more_link'})
            for link in review_links:
                url_review = link.get('href')
                print(url_review)
                
                review_id = os.path.basename(url_review)
                index_questionmark = review_id.index('?')
                review_id = review_id[:index_questionmark]
                headers = {'User-Agent': select_agent() }
                response_review = requests.get(url_review,headers=headers)
                html_page2 = response_review.text
                soup = BeautifulSoup( html_page2 ,"lxml")

                review_text = soup.find('div',{'class':'reviewText'})
                full_text = ''
                if review_text is not None:
                    full_text = review_text.text 
                    full_text = re.sub('\s+',' ',full_text)

                language = 'Unknown'
                try:
                    language = detect(full_text)
                except:
                    print('Lanuage could not be detected')

                if language =='en':
                    out.write(f'{review_id}\t{full_text.strip()}\n')

            i+=1

        out.close()

https://www.goodreads.com/book/isbn/?key=QypW6JzQ4XfGViss1Ks4g&id=9781538761465&format=json&text_only=true
reviews_widget
{'reviews_widget': '<style>\n  #goodreads-widget {\n    font-family: georgia, serif;\n    padding: 18px 0;\n    width:565px;\n  }\n  #goodreads-widget h1 {\n    font-weight:normal;\n    font-size: 16px;\n    border-bottom: 1px solid #BBB596;\n    margin-bottom: 0;\n  }\n  #goodreads-widget a {\n    text-decoration: none;\n    color:#660;\n  }\n  iframe{\n    background-color: #fff;\n  }\n  #goodreads-widget a:hover { text-decoration: underline; }\n  #goodreads-widget a:active {\n    color:#660;\n  }\n  #gr_footer {\n    width: 100%;\n    border-top: 1px solid #BBB596;\n    text-align: right;\n  }\n  #goodreads-widget .gr_branding{\n    color: #382110;\n    font-size: 11px;\n    text-decoration: none;\n    font-family: "Helvetica Neue", Helvetica, Arial, sans-serif;\n  }\n</style>\n<div id="goodreads-widget">\n  <div id="gr_header"><h1><a rel="nofollow" href="https:/

KeyboardInterrupt: 

In [ ]:
#https://www.googleapis.com/books/v1/volumes?q=isbn:9781451648546